In [1]:
#%pip install --quiet PyYAML

In [2]:
minio_keys_file="../secrets/minio.keys"

In [3]:
! cat ../secrets/minio.keys

access_key: ehRjBCNcqOqcW6U2bJlV
secret_key: 0tiCzra8npr4K4snnBQBpoXUI0fA1qvP96XnE9oF


In [4]:
import yaml
minio_conf=None
with open(minio_keys_file) as f:
    minio_conf = yaml.safe_load(f)
#minio_conf["access_key"], minio_conf["secret_key"], 

In [5]:
## DEFINE SENSITIVE VARIABLES
NESSIE_URI = "http://nessie:19120/api/v1"
MINIO_ACCESS_KEY = minio_conf["access_key"]
MINIO_SECRET_KEY = minio_conf["secret_key"]
MINIO_ENDPOINT_URL = "http://minio:9000"

In [6]:
MINIO_ACCESS_KEY,MINIO_SECRET_KEY

('ehRjBCNcqOqcW6U2bJlV', '0tiCzra8npr4K4snnBQBpoXUI0fA1qvP96XnE9oF')

In [14]:
import pyspark
from pyspark.sql import SparkSession
import os





conf = (
        pyspark.SparkConf()
        .setAppName('app_name')
        .set(
                'spark.jars.packages',
                'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,'
                'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,'
                'software.amazon.awssdk:bundle:2.17.178,'
                'software.amazon.awssdk:url-connection-client:2.17.178'
        ).set(
                'spark.sql.extensions',
                'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,'
                'org.projectnessie.spark.extensions.NessieSparkSessionExtensions'
        ).set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
        .set('spark.sql.catalog.nessie.s3.endpoint', MINIO_ENDPOINT_URL)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
        .set('spark.sql.catalog.nessie.s3.access-key-id', MINIO_ACCESS_KEY)
        .set('spark.sql.catalog.nessie.s3.secret-access-key', MINIO_SECRET_KEY)
        .set('spark.sql.catalog.nessie.s3.path-style-access', 'true')
        )


# Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
csv_df.createOrReplaceTempView("csv_open_2023")
spark.sql("CREATE TABLE IF NOT EXISTS nessie.df_open_2023_lesson2 USING iceberg AS SELECT * FROM csv_open_2023").show()
spark.sql("SELECT * FROM nessie.df_open_2023_lesson2 limit 10").show()
spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()
spark.sql("CREATE BRANCH IF NOT EXISTS lesson2 IN nessie")
spark.sql("USE REFERENCE lesson2 IN nessie")
spark.sql("DELETE FROM nessie.df_open_2023_lesson2 WHERE countryOfOriginCode = 'FR'")
spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()
spark.sql("USE REFERENCE main IN nessie")
spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()
spark.sql("MERGE BRANCH lesson2 INTO main IN nessie")
spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()



Spark Running
++
||
++
++

+------------+------------------+---------+--------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|    competitorName|firstName|lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+------------------+---------+--------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|     2455538|    Siobhan Tenner|  Siobhan|  Tenner|   ACT|     F|                 ZA|       South Africa|      30|            Africa|      11447|CrossFit Richards...| 25|150 cm| 63 kg|      64862|      256308|       2|2023|
|     2423044|      Saioa Palomo|    Saioa|  Palomo|   ACT|     F|       

In [13]:
!head -4 ../datasets/df_open_2023.csv




competitorId,competitorName,firstName,lastName,status,gender,countryOfOriginCode,countryOfOriginName,regionId,regionName,affiliateId,affiliateName,age,height,weight,overallRank,overallScore,genderId,year
469656,Jeffrey Adler,Jeffrey,Adler,ACT,M,CA,Canada,35,North America East,18059,CrossFit Wonderland,29,69 in,197 lb,1,107,1,2023
300638,Tola Morakinyo,Tola,Morakinyo,ACT,M,US,United States,35,North America East,19593,CrossFit East Nashville,27,72 in,212 lb,2,219,1,2023
676693,Colten Mertens,Colten,Mertens,ACT,M,US,United States,34,North America West,1324,CrossFit Kilo,25,64 in,180 lb,3,367,1,2023
